In [1]:
# example_meta_data_reader.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to deal with the output files

In [2]:
%matplotlib qt
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import h5py


In [3]:
example_set = ak.from_parquet('./output/custom_1337_meta_data.parquet')

In [4]:
example_set.fields

['event_id', 'mc_truth', 'primary_lepton_1', 'primary_hadron_1', 'total']

In [5]:
example_set.mc_truth.fields

['injection_energy',
 'injection_type',
 'injection_interaction_type',
 'injection_zenith',
 'injection_azimuth',
 'injection_bjorkenx',
 'injection_bjorkeny',
 'injection_position_x',
 'injection_position_y',
 'injection_position_z',
 'injection_column_depth',
 'primary_lepton_1_type',
 'primary_hadron_1_type',
 'primary_lepton_1_position_x',
 'primary_lepton_1_position_y',
 'primary_lepton_1_position_z',
 'primary_hadron_1_position_x',
 'primary_hadron_1_position_y',
 'primary_hadron_1_position_z',
 'primary_lepton_1_direction_theta',
 'primary_lepton_1_direction_phi',
 'primary_hadron_1_direction_theta',
 'primary_hadron_1_direction_phi',
 'primary_lepton_1_energy',
 'primary_hadron_1_energy',
 'total_energy']

In [14]:
example_set.total.fields

['sensor_id',
 'sensor_pos_x',
 'sensor_pos_y',
 'sensor_pos_z',
 'sensor_string_id',
 't']

In [6]:
for field in example_set.mc_truth.fields:
    print(field)
    print(example_set.mc_truth[field])

injection_energy
[1.14e+03, 1.7e+03, 1.18e+03, 7.1e+03, ... 1.68e+03, 1.06e+03, 3.98e+03, 1.3e+03]
injection_type
[14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
injection_interaction_type
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
injection_zenith
[2.35, 1.79, 2.25, 1.74, 2.85, 2.59, 2.37, 1.71, 2.01, 3.09]
injection_azimuth
[4.93, 4.39, 3.37, 1.99, 5.28, 2.26, 0.548, 1.7, 6.28, 1.04]
injection_bjorkenx
[0.288, 0.595, 0.31, 0.391, 0.0247, 0.338, 0.169, 0.0289, 0.452, 0.0606]
injection_bjorkeny
[0.288, 0.595, 0.31, 0.391, 0.0247, 0.338, 0.169, 0.0289, 0.452, 0.0606]
injection_position_x
[-138, -239, 164, 2.4, -183, -228, -153, 177, 104, -157]
injection_position_y
[172, 17, -49.8, 72.6, 20.3, 29.4, -169, -18.5, -88.3, -9.23]
injection_position_z
[333, -334, -515, -595, 356, -405, 263, 252, -476, -46.5]
injection_column_depth
[6.2e+04, 2.13e+04, 2.38e+04, 3.28e+04, ... 6.57e+04, 3.56e+04, 4.94e+04, 1.11e+05]
primary_lepton_1_type
[13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
primary_hadron_1_type
[-2000001006, 

In [7]:
example_set.primary_hadron_1.fields

['sensor_id',
 'sensor_pos_x',
 'sensor_pos_y',
 'sensor_pos_z',
 'sensor_string_id',
 't',
 'loss_pos_x',
 'loss_pos_y',
 'loss_pos_z',
 'loss_n_photons']

In [8]:
example_set.mc_truth.primary_lepton_1_position_x

<Array [-138, -239, 164, ... 177, 104, -157] type='10 * float64'>

In [9]:
example_set.event_id

<Array [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] type='10 * int64'>

In [10]:
# The events are split into three parts:
#   - lepton: the light from the generated lepton
#   - hadron: the light produced from the associated hadronic cascade (at production point)
#   - total: the light from both parts
# Each of these fields is formatted the same way
example_set.fields

['event_id', 'mc_truth', 'primary_lepton_1', 'primary_hadron_1', 'total']

In [11]:
# Example for one event
event_id = 1
# The sensors hit
print("Total number of hits:")
print(len(example_set.total.sensor_id[event_id]))
print('The hit sensors: ')
print(example_set.total.sensor_id[event_id])
print('Their associated strings')
print(example_set.total.sensor_string_id[event_id])
print('The hit times')
print(example_set.total.t[event_id])

Total number of hits:
0
The hit sensors: 
[]
Their associated strings
[]
The hit times
[]


In [12]:
def paqruet_plotting(
        data, e_id,
        brightest_event=False,
        channel='total',
        show_lepton=True,
        scaling=20
    ):
    try:
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        hit_counts = []
        if brightest_event:
            for event in data[channel].sensor_pos_x:
                hit_counts.append(len(event))
            hit_counts = np.array(hit_counts)
            event_id = np.argmax(hit_counts)
            print('The brightest event has the id %d' % event_id)
            print('The energy of the injected particle is %.1f' % data.mc_truth.injection_energy[event_id])
        else:
            event_id = e_id
        sensor_comb = np.array([
            [data[channel].sensor_pos_x[event_id][i],
            data[channel].sensor_pos_y[event_id][i],
            data[channel].sensor_pos_z[event_id][i]
            ]
            for i in range(len(data[channel].sensor_pos_x[event_id]))
        ])
        hit_doms, hit_counts = np.unique(sensor_comb, axis=0, return_counts=True)
        ax.scatter(
            hit_doms[:, 0],
            hit_doms[:, 1],
            hit_doms[:, 2],
            s=(
                hit_counts / max(hit_counts) * scaling
            ),
            color='r', alpha=0.5)
        if show_lepton:
            ax.scatter(
                data.primary_lepton_1.loss_pos_x[event_id],
                data.primary_lepton_1.loss_pos_y[event_id],
                data.primary_lepton_1.loss_pos_z[event_id],
                s=(
                    data.primary_lepton_1.loss_n_photons[event_id] /
                    max(data.primary_lepton_1.loss_n_photons[event_id]) *
                    scaling
                ),
                color='blue', alpha=0.5)
        ax.set_xlim(min(hit_doms[:, 0]), max(hit_doms[:, 0]))
        ax.set_ylim(min(hit_doms[:, 1]), max(hit_doms[:, 1]))
        ax.set_zlim(min(hit_doms[:, 2]), max(hit_doms[:, 2]))
        ax.set_xlabel('X [m]')
        ax.set_ylabel('Y [m]')
        ax.set_zlabel('Z [m]')
        plt.title(
            'Event %d with pdg id = %d and energy %.1f GeV' % (
                event_id,
                data.mc_truth.primary_lepton_1_type[event_id],
                data.mc_truth.primary_lepton_1_energy[event_id]
                )
        )
    except:
        print('No hits found!')

In [24]:
paqruet_plotting(
    example_set,
    e_id=4,
    brightest_event=True,
    channel='total',
    show_lepton=True,
    scaling=100
)

The brightest event has the id 6
The energy of the injected particle is 1685.0


/tmp/ipykernel_22643/2656468167.py:48: UserWarning: Attempting to set identical left == right == 50.0 results in singular transformations; automatically expanding.
  ax.set_xlim(min(hit_doms[:, 0]), max(hit_doms[:, 0]))
/tmp/ipykernel_22643/2656468167.py:49: UserWarning: Attempting to set identical bottom == top == -28.867513459481298 results in singular transformations; automatically expanding.
  ax.set_ylim(min(hit_doms[:, 1]), max(hit_doms[:, 1]))
